<a href="https://colab.research.google.com/github/thorrester/Basic_Tensorflow/blob/master/colab_keras_imbalance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass


TensorFlow 2.x selected.


In [8]:
import os
os.getcwd()

'/content'

In [10]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [17]:
!ls "/content/drive/My Drive/data"

creditcard.csv


In [0]:
path = "/content/drive/My Drive/data"

In [22]:

import csv
import numpy as np

# Get the real data from https://www.kaggle.com/mlg-ulb/creditcardfraud/downloads/creditcardfraud.zip/
fname = path + '/creditcard.csv'

all_features = []
all_targets = []
with open(fname) as f:
  for i, line in enumerate(f):
    if i == 0:
      print('HEADER:', line.strip())
      continue  # Skip header
    fields = line.strip().split(',')
    all_features.append([float(v.replace('"', '')) for v in fields[:-1]])
    all_targets.append([int(fields[-1].replace('"', ''))])
    if i == 1:
      print('EXAMPLE FEATURES:', all_features[-1])

features = np.array(all_features, dtype='float32')
targets = np.array(all_targets, dtype='uint8')
print('features.shape:', features.shape)
print('targets.shape:', targets.shape)

HEADER: Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
EXAMPLE FEATURES: [0.0, -1.359807134, -0.072781173, 2.536346738, 1.378155224, -0.33832077, 0.462387778, 0.239598554, 0.098697901, 0.36378697, 0.090794172, -0.551599533, -0.617800856, -0.991389847, -0.311169354, 1.468176972, -0.470400525, 0.207971242, 0.02579058, 0.40399296, 0.251412098, -0.018306778, 0.277837576, -0.11047391, 0.066928075, 0.128539358, -0.189114844, 0.133558377, -0.021053053, 149.62]
features.shape: (284807, 30)
targets.shape: (284807, 1)


In [0]:
num_val_samples = int(len(features) * 0.2)
train_features = features[:-num_val_samples]
train_targets = targets[:-num_val_samples]
val_features = features[-num_val_samples:]
val_targets = targets[-num_val_samples:]

In [26]:
print('Number of training samples:', len(train_features))
print('Number of validation samples:', len(val_features))

counts = np.bincount(train_targets[:, 0])
print('Number of positive samples in training data: {} ({:.2f}% of total)'.format(counts[1], 100 * float(counts[1]) / len(train_targets)))

weight_for_0 = 1. / counts[0]
weight_for_1 = 1. / counts[1]
print(weight_for_0)
print(weight_for_1)

mean = np.mean(train_features, axis=0)
train_features -= mean
val_features -= mean
std = np.std(train_features, axis=0)
train_features /= std
val_features /= std

from tensorflow import keras

model = keras.Sequential([
  keras.layers.Dense(256, activation='relu',
                     input_shape=(train_features.shape[-1],)),
  keras.layers.Dense(256, activation='relu'),
  keras.layers.Dropout(0.3),
  keras.layers.Dense(256, activation='relu'),
  keras.layers.Dropout(0.3),
  keras.layers.Dense(1, activation='sigmoid'),
])
model.summary()

Number of training samples: 227846
Number of validation samples: 56961
Number of positive samples in training data: 417 (0.18% of total)
4.396976638863118e-06
0.002398081534772182
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 256)               7936      
_________________________________________________________________
dense_5 (Dense)              (None, 256)               65792     
_________________________________________________________________
dropout_2 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 256)               65792     
_________________________________________________________________
dropout_3 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_

In [27]:
metrics = [keras.metrics.FalseNegatives(name='fn'),
           keras.metrics.FalsePositives(name='fp'),
           keras.metrics.TrueNegatives(name='tn'),
           keras.metrics.TruePositives(name='tp'),
           keras.metrics.Precision(name='precision'),
           keras.metrics.Recall(name='recall')]

model.compile(optimizer=keras.optimizers.Adam(1e-2),
              loss='binary_crossentropy',
              metrics=metrics)

callbacks = [keras.callbacks.ModelCheckpoint('fraud_model_at_epoch_{epoch}.h5')]
class_weight = {0: weight_for_0, 1: weight_for_1}

model.fit(train_features, train_targets,
          batch_size=2048,
          epochs=50,
          verbose=2,
          callbacks=callbacks,
          validation_data=(val_features, val_targets)
          ,class_weight=class_weight)

Train on 227846 samples, validate on 56961 samples
Epoch 1/50
227846/227846 - 3s - loss: 2.2383e-06 - fn: 45.0000 - fp: 27451.0000 - tn: 199978.0000 - tp: 372.0000 - precision: 0.0134 - recall: 0.8921 - val_loss: 1.2841e-06 - val_fn: 11.0000 - val_fp: 254.0000 - val_tn: 56632.0000 - val_tp: 64.0000 - val_precision: 0.2013 - val_recall: 0.8533
Epoch 2/50
227846/227846 - 1s - loss: 1.4847e-06 - fn: 36.0000 - fp: 6970.0000 - tn: 220459.0000 - tp: 381.0000 - precision: 0.0518 - recall: 0.9137 - val_loss: 1.0810e-06 - val_fn: 9.0000 - val_fp: 1506.0000 - val_tn: 55380.0000 - val_tp: 66.0000 - val_precision: 0.0420 - val_recall: 0.8800
Epoch 3/50
227846/227846 - 1s - loss: 1.3318e-06 - fn: 34.0000 - fp: 8600.0000 - tn: 218829.0000 - tp: 383.0000 - precision: 0.0426 - recall: 0.9185 - val_loss: 1.0501e-06 - val_fn: 9.0000 - val_fp: 750.0000 - val_tn: 56136.0000 - val_tp: 66.0000 - val_precision: 0.0809 - val_recall: 0.8800
Epoch 4/50
227846/227846 - 1s - loss: 9.7602e-07 - fn: 25.0000 - fp: 6